In [ ]:
import numpy as np

In [ ]:
class Option:
    
    def __init__(self, stock_price, strike, vol, rfr, years):
        self.stock_price = stock_price
        self.strike = strike
        self.vol = vol
        self.rfr = rfr
        self.years = years

In [ ]:
class BinomialTree:
    
    def __init__(self, opt: Option, steps: int):
        self.opt = opt
        self.steps = steps
        self.tree = self.opt.stock_price*self.factors()[0]**np.arange(-steps, steps+1, 2)[::-1]
        
    def factors(self):
        steps = self.steps
        s = self.opt.vol
        t = self.opt.years
        up = np.exp(s*np.sqrt(t/steps))
        down = 1/up
        return up, down
    
    def probabilities(self):
        u, d = self.factors()
        steps = self.steps
        r = self.opt.rfr
        t = self.opt.years
        up = (np.exp(r*(t/steps))-d)/(u-d)
        down = 1 - up
        return np.array([up, down])
    
    def PV_AMER(self, typeof: str):
        K = self.opt.strike
        r = self.opt.rfr
        T = self.opt.years
        steps = self.steps
        tree = self.tree
        u, d = self.factors()
        prob = self.probabilities()[::-1]
        
        if typeof == 'call':
            payoff = np.maximum(self.tree-K, np.zeros(steps+1))
        else:
            payoff = np.maximum(K-self.tree, np.zeros(steps+1))
        
        for i in range(steps):
            cont_value = np.convolve(payoff, prob, 'valid')*np.exp(-r*(T/steps))
            curr_layer = tree[1:]/d
            if typeof == 'call':
                exer_value = np.maximum(curr_layer-K, np.zeros(steps-i))
            else:
                exer_value = np.maximum(K-curr_layer, np.zeros(steps-i))
            
            payoff = np.maximum(cont_value, exer_value)
            tree = curr_layer
        
        return round(payoff[0], 2)